In [1]:
import os

In [2]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\DataScience\\Projects\\Phishing_Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class PredictionConfig:
    model_path: Path

In [7]:
!pip install python-whois

In [8]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [9]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import networkx as nx
import joblib
import numpy as np
import pandas as pd

In [10]:
from Phishingproject.utils.common import *
from Phishingproject.constants import *

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        self.schema = read_yaml_file(schema_filepath)

    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.prediction
        
        prediction_config = PredictionConfig(
            model_path = config.model_path  
        )

        return prediction_config

In [12]:
class PredictionPipeline:
    def __init__(self,config:PredictionConfig):
        self.config = config

    def featureExtraction(self,url):
        features = []
        #Address bar based features (10)
        features.append(havingIP(url))
        features.append(haveAtSign(url))
        features.append(getLength(url))
        features.append(getDepth(url))
        features.append(redirection(url))
        features.append(httpDomain(url))
        features.append(tinyURL(url))
        features.append(prefixSuffix(url))
        #Domain based features (4)
        dns = 0
        try:
          domain_name = whois.whois(urlparse(url).netloc)
        except:
          dns = 1

        features.append(dns)
        features.append(is_potential_phishing(url))
        features.append(1 if dns == 1 else domainAge(domain_name))
        features.append(1 if dns == 1 else domainEnd(domain_name))

        # HTML & Javascript based features (4)
        try:
          response = requests.get(url)
        except:
          response = ""
        features.append(iframe(response))
        features.append(mouseOver(response))
        features.append(rightClick(response))
        features.append(forwarding(response))

        print(features)
        print(type(features))
        print(len(features))

        return features
    
    def predict(self,url):
        model=joblib.load(self.config.model_path)
        data=self.featureExtraction(url)
        # Get the feature names from the model
        if hasattr(model, 'feature_names_in_'):
          feature_names = model.feature_names_in_
        else:
        # If feature names are not available, create generic names
          feature_names = [f'feature_{i}' for i in range(len(data))]
        print(feature_names)
        # Create a pandas DataFrame with the correct feature names
        input_df = pd.DataFrame([data], columns=feature_names)
        pred=int((model.predict(input_df))[0])
        # pred=int(pred[0])
        print(type(pred))
        print(pred)


In [13]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction= PredictionPipeline(config=prediction_config)
    PW=prediction.predict("http://emgn.com/tv/schnauzer-passes-out-with-excitement-when-he-is-reunited-with-his-human-after-2-years/")
except Exception as e:
    raise e

[2024-07-11 17:50:51,819: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-07-11 17:50:51,819: INFO:common: yaml file: params.yaml loaded successfully]
[2024-07-11 17:50:51,827: INFO:common: yaml file: schema.yaml loaded successfully]
Error occurred while evaluating http://emgn.com/tv/schnauzer-passes-out-with-excitement-when-he-is-reunited-with-his-human-after-2-years/: HTTPConnectionPool(host='emgn.com', port=80): Max retries exceeded with url: /tv/schnauzer-passes-out-with-excitement-when-he-is-reunited-with-his-human-after-2-years/ (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x000001349CF923A0>: Failed to resolve 'emgn.com' ([Errno 11002] getaddrinfo failed)"))
[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
<class 'list'>
16
['Have_IP' 'Have_At' 'URL_Length' 'URL_Depth' 'Redirection' 'https_Domain'
 'TinyURL' 'Prefix/Suffix' 'DNS_Record' 'Web_Traffic' 'Domain_Age'
 'Domain_End' 'iFrame' 'Mouse_Over' 'Right_Click' 'Web_Forwards